In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from scipy.stats import pearsonr
from collections import Counter

<IPython.core.display.Javascript object>

In [3]:
def convert_to_float(value):
    try:
        # If it's a string enclosed in single quotes, remove quotes and replace comma with dot
        if isinstance(value, str):
            return float(value.replace(",", "."))
        else:
            return float(value)
    except ValueError as e:
        print(e)
        return float("nan")

<IPython.core.display.Javascript object>

In [4]:
def preprocess_numeric_cols(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)  # Ensure string type for string operations
        df[col] = df[col].str.replace("#", "")  # Remove '#' characters
        df[col] = df[col].str.replace(",", ".")  # Replace ',' with '.' for decimals
    return df

<IPython.core.display.Javascript object>

# Data preparation - 204 plant F data

### Reading the files and extracting relevant information:

In [5]:
plant = "F"
xls_files = {}


xls = pd.ExcelFile(
    f"../../../../data/raw/204/EMBRAPII hubIC IACC CIMENTO_{plant}.xlsx",
    engine="openpyxl",
)
xls_files[plant] = xls

<IPython.core.display.Javascript object>

In [6]:
print("Plant: ", plant, xls.sheet_names)

Plant:  F ['INSTRUÇÕES', '55', '75', '14', '91', '65']


<IPython.core.display.Javascript object>

In [7]:
cement_types_per_plant = []
CEMENT_TYPE_COLS = ["Tipo de cimento", "Classe de resistência"]
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[1:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[1, 2])
        if not df[CEMENT_TYPE_COLS].iloc[2:].isna().all().all():
            cement_type = (
                df[CEMENT_TYPE_COLS]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .str.replace("nan", "")
                .unique()
            )

        else:
            cement_type = (
                df[[("Obs.", "Unnamed: 8_level_1")]]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .unique()
            )

        cement_types_per_plant.append((plant, sheet_name, cement_type))
        dataframes.append(df)

<IPython.core.display.Javascript object>

In [8]:
cement_types_per_plant

[('F', '55', array(['CPVARI'], dtype=object)),
 ('F', '75', array(['CPIIE32'], dtype=object)),
 ('F', '14', array(['CPIIF32', ''], dtype=object)),
 ('F', '91', array(['CPIV32'], dtype=object)),
 ('F', '65', array(['CPVARIRS'], dtype=object))]

<IPython.core.display.Javascript object>

In [9]:
cements = []
for tup in cement_types_per_plant:
    cements.append(tup[2][0])
Counter(cements)

Counter({'CPVARI': 1, 'CPIIE32': 1, 'CPIIF32': 1, 'CPIV32': 1, 'CPVARIRS': 1})

<IPython.core.display.Javascript object>

In [10]:
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[1:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[0, 1, 2])
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"] = plant
        dataframes.append(df.drop([0, 1], axis=0).reset_index(drop=True).copy())

<IPython.core.display.Javascript object>

In [11]:
df = pd.concat(dataframes, axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

## Initial Preprocessing

In [12]:
df = pd.concat(
    [
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"],
        df["Dados iniciais"],
        df["Cimento"]["Análise química"],
        df["Característias físicas do cimento"],
        df["Propriedades do cimento"],
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

### 1. Dropping columns where either there is no data nor it contributes to the analysis
### 2. Renaming columns to a short identifiable name

In [13]:
COLUMNS_TO_DROP = [
    ("Data", "Medida"),
    ("Horário", "Produção"),
    ("Horário", "Medida"),
    ("Obs.", "Unnamed: 148_level_2"),
    ("Tipo de cimento", "Unnamed: 6_level_2"),
    ("Classe de resistência", "Unnamed: 7_level_2"),
    ("Obs.", "Unnamed: 8_level_2"),
    "Obs.",
    ("Obs.", "Unnamed: 148_level_2"),
    ("Unnamed: 141_level_1", "Obs."),
]

COLUMNS_TO_RENAME = {
    ("Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"): "Plant",
    ("Cidade", "Unnamed: 1_level_2"): "City",
    ("Data", "Produção"): "Date",  # Arrumar!
    ("Unnamed: 136_level_1", "Blaine"): "Blaine",
    ("Unnamed: 137_level_1", "#200"): "#200",
    ("Unnamed: 138_level_1", "#325"): "#325",
    ("Unnamed: 139_level_1", "#400"): "#400",
    ("Unnamed: 140_level_1", "Massa específica"): "Specific Gravity",
    ("Tempo de pega", "Fim"): "Final setting time",
    ("Tempo de pega", "Inicio"): "Initial setting time",
    ("Resistência à compressão", "1 d"): "CS1",
    ("Resistência à compressão", "3 d"): "CS3",
    ("Resistência à compressão", "7 d"): "CS7",
    ("Resistência à compressão", "28 d"): "CS28",
    "Perda ao fogo": "Loss on Ignition",
    "Resíduo insolúvel": "Insoluble Residue",
}

<IPython.core.display.Javascript object>

### Defining a single variable to identify Cement Type

In [14]:
# Single variable for the Cement Type
df["Cement_Type"] = (
    df[
        [
            ("Tipo de cimento", "Unnamed: 6_level_2"),
            ("Classe de resistência", "Unnamed: 7_level_2"),
            ("Obs.", "Unnamed: 8_level_2"),
        ]
    ]
    .fillna("")
    .astype(str)
    .sum(axis=1)
)

# # Preprocessing
df["Cement_Type"] = df["Cement_Type"].replace("Fibro", "Fibrocimento")

<IPython.core.display.Javascript object>

### Dropping and rename the columns defined above

In [15]:
df = df.drop(COLUMNS_TO_DROP, axis=1).rename(COLUMNS_TO_RENAME, axis=1).copy()
df = df.drop([0]).reset_index(drop=True)
df = df.infer_objects()

<IPython.core.display.Javascript object>

### Cleaning and converting numeric columns to float

In [16]:
NUMERIC_COLUMNS = [
    "CaO",
    "MgO",
    "Na2O",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "TiO2",
    "Fe2O3",
    "Loss on Ignition",
    "Insoluble Residue",
    "Blaine",
    "#200",
    "#325",
    "#400",
    "Specific Gravity",
    "Final setting time",
    "Initial setting time",
    "CS1",
    "CS3",
    "CS7",
    "CS28",
]

df = preprocess_numeric_cols(df, NUMERIC_COLUMNS)

# Preprocessing to fix numeric columns
df[NUMERIC_COLUMNS] = df[NUMERIC_COLUMNS].map(convert_to_float)

# Converting Date to pandas datetime
df["Date"] = pd.to_datetime(df["Date"])

# Drop rows completely missing
index_to_keep = (
    df.drop(["Plant", "Date", "Cement_Type"], axis=1).dropna(axis=0, how="all").index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

df = df.infer_objects()

<IPython.core.display.Javascript object>

## Dropping missing features

### Drop rows completely missing

In [17]:
index_to_keep = (
    df.drop(["Plant", "City", "Date", "Cement_Type"], axis=1)
    .dropna(axis=0, how="all")
    .index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

<IPython.core.display.Javascript object>

### Removing features
 that will be fitted on this data.

1. Removing features with features having ~50% or more of missing values
2. Removing features with zero variance
3. Removing features with more than 70% of zeros

<h3>Percentage of missing values:</h3>
    
    
<table>
    <th>Plant AQ - Full Dataset:</th>
    <th></th>
    <tr>
        <td>Specific Gravity</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>TiO2</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>#400</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>Insoluble Residue</td>
        <td>91.85</td>
    </tr>    
</table>

In [18]:
FEATRUES_TO_DROP = ["Specific Gravity", "Insoluble Residue", "TiO2", "#400"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping variable with City and Plant identification

In [19]:
FEATRUES_TO_DROP = ["City", "Plant"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is null

In [20]:
df = df[df["CS28"].notna()]

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is zero

In [21]:
df = df[~df["CS28"].eq(0)]

<IPython.core.display.Javascript object>

### Dropping rows where any other variable has a zero value

In [22]:
for col in df.columns:
    df = df[~df[col].eq(0)]

<IPython.core.display.Javascript object>

### Dropping duplicated rows

In [23]:
df.shape

(2562, 20)

<IPython.core.display.Javascript object>

In [24]:
df = df.reset_index(drop=True)

<IPython.core.display.Javascript object>

In [25]:
df = df.loc[df.select_dtypes("number").drop_duplicates().index].reset_index(drop=True)

chemical_vars = [
    "CaO",
    "MgO",
    "Na2O",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "Fe2O3",
    "Loss on Ignition",
]

df = df.loc[
    df.select_dtypes("number").drop_duplicates(subset=chemical_vars).index
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [26]:
df.shape

(2549, 20)

<IPython.core.display.Javascript object>

### Dropping rows with negative values

In [27]:
df[df.drop(["Date"], axis=1).duplicated()].drop("Date", axis=1).select_dtypes(
    include="number"
).apply(lambda x: x < 0).sum().sort_values(ascending=False)

CaO                     0.0
MgO                     0.0
CS7                     0.0
CS3                     0.0
CS1                     0.0
Final setting time      0.0
Initial setting time    0.0
#325                    0.0
#200                    0.0
Blaine                  0.0
Loss on Ignition        0.0
Fe2O3                   0.0
K2O                     0.0
SO3                     0.0
SiO2                    0.0
Al2O3                   0.0
Na2O                    0.0
CS28                    0.0
dtype: float64

<IPython.core.display.Javascript object>

### Removing outliers

In [28]:
df.describe(percentiles=[0.99]).T

,count,mean,min,50%,99%,max,std
Date,2549,2021-10-11 23:45:52.608866304,2020-01-02 00:00:00,2021-10-17 00:00:00,2023-06-19 12:28:48,2023-07-02 00:00:00,NaN
CaO,2546.0,61.272121,2.68,60.8,67.161,69.59,4.201216
MgO,2546.0,2.932317,0.69,2.86,4.3755,6.06,0.53125
Na2O,2546.0,0.225609,0.05,0.22,0.681,0.97,0.085157
Al2O3,2546.0,4.160499,0.12,4.43,5.872,21.85,1.677175
SiO2,2546.0,20.488755,2.01,20.53,24.6355,66.27,2.344189
SO3,2546.0,3.469042,1.55,3.88,4.45,4.53,0.801044
K2O,2546.0,0.895683,0.12,0.89,1.02,4.31,0.152091
Fe2O3,2546.0,2.487636,1.41,2.49,3.32,5.83,0.237584
Loss on Ignition,2548.0,5.463375,2.29,4.73,12.1706,12.5,2.101247


<IPython.core.display.Javascript object>

In [29]:
df = df[df["Al2O3"] < 8]
df = df[df["SiO2"] < 25]
df = df[df["K2O"] < 2]
df = df[df["#325"] < 20]

<IPython.core.display.Javascript object>

In [30]:
df.describe(percentiles=[0.99]).T

,count,mean,min,50%,99%,max,std
Date,2514,2021-10-13 11:36:30.930787584,2020-01-02 00:00:00,2021-10-18 12:00:00,2023-06-19 20:52:48,2023-07-02 00:00:00,NaN
CaO,2514.0,61.297613,2.68,60.83,67.1674,69.59,4.189881
MgO,2514.0,2.928632,0.69,2.86,4.36,6.06,0.529828
Na2O,2514.0,0.225684,0.05,0.22,0.5522,0.97,0.084312
Al2O3,2514.0,4.126746,0.12,4.43,5.82,7.14,1.444205
SiO2,2514.0,20.431559,2.01,20.51,24.3687,24.95,2.034196
SO3,2514.0,3.475708,1.55,3.88,4.45,4.53,0.797387
K2O,2514.0,0.889694,0.12,0.89,1.02,1.16,0.078408
Fe2O3,2514.0,2.486881,1.41,2.49,3.3187,5.83,0.237405
Loss on Ignition,2514.0,5.461293,2.29,4.72,12.1774,12.5,2.108679


<IPython.core.display.Javascript object>

In [31]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
CS1,32.736675
CS7,0.039777
CS3,0.039777
Date,0.000000
CaO,0.000000
CS28,0.000000
Final setting time,0.000000
Initial setting time,0.000000
#325,0.000000
#200,0.000000


<IPython.core.display.Javascript object>

In [32]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Date,2514,2021-10-13 11:36:30.930787584,2020-01-02 00:00:00,2020-12-17 06:00:00,2021-10-18 12:00:00,2022-07-31 18:00:00,2023-07-02 00:00:00,NaN
CaO,2514.0,61.297613,2.68,58.9325,60.83,64.9475,69.59,4.189881
MgO,2514.0,2.928632,0.69,2.55,2.86,3.25,6.06,0.529828
Na2O,2514.0,0.225684,0.05,0.19,0.22,0.26,0.97,0.084312
Al2O3,2514.0,4.126746,0.12,4.1625,4.43,5.09,7.14,1.444205
SiO2,2514.0,20.431559,2.01,19.47,20.51,21.61,24.95,2.034196
SO3,2514.0,3.475708,1.55,2.62,3.88,4.06,4.53,0.797387
K2O,2514.0,0.889694,0.12,0.85,0.89,0.94,1.16,0.078408
Fe2O3,2514.0,2.486881,1.41,2.36,2.49,2.57,5.83,0.237405
Loss on Ignition,2514.0,5.461293,2.29,4.07,4.72,6.15,12.5,2.108679


<IPython.core.display.Javascript object>

In [33]:
df.describe(exclude="number").T

,count,unique,top,freq,mean,min,25%,50%,75%,max
Date,2514,NaN,NaN,NaN,2021-10-13 11:36:30.930787584,2020-01-02 00:00:00,2020-12-17 06:00:00,2021-10-18 12:00:00,2022-07-31 18:00:00,2023-07-02 00:00:00
Cement_Type,2514,5,CPV ARI,942,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2514 entries, 0 to 2548
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  2514 non-null   datetime64[ns]
 1   CaO                   2514 non-null   float64       
 2   MgO                   2514 non-null   float64       
 3   Na2O                  2514 non-null   float64       
 4   Al2O3                 2514 non-null   float64       
 5   SiO2                  2514 non-null   float64       
 6   SO3                   2514 non-null   float64       
 7   K2O                   2514 non-null   float64       
 8   Fe2O3                 2514 non-null   float64       
 9   Loss on Ignition      2514 non-null   float64       
 10  Blaine                2514 non-null   float64       
 11  #200                  2514 non-null   float64       
 12  #325                  2514 non-null   float64       
 13  Initial setting time  2

<IPython.core.display.Javascript object>

### Sort the dataset by date

In [35]:
# We first make sure Date column was infered correctly
df["Date"].dtype

dtype('<M8[ns]')

<IPython.core.display.Javascript object>

In [36]:
df = df.sort_values(by="Date")

<IPython.core.display.Javascript object>

## Saving the dataset

<h3>Saving for feature engineering</h3>

In [37]:
df.to_csv("../../../../data/interim/204/f.csv", index=False)

<IPython.core.display.Javascript object>